This notebook contains the code used for the conversion of all gene names used throughout the pipeline to Gencode v.34. 

We convert: 

1. List of recessive genes

2. s-hets from Cassa et al.

3. s-hets from Weghorn et al. 

4. pLI from gnomad

5. Roulette 

In [ ]:
import pandas as pd
from pybiomart import Dataset

In [ ]:
def get_gencode_v34_HGNC_genes():
    """
        This function dowloads gene name and its HGNC id as specified in gencode v.34
    """
    # load inhouse

    inhouse = pd.read_csv("https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_34/gencode.v34.metadata.HGNC.gz", 
                          compression='gzip', sep='\t', header=None)
    inhouse.columns = ['Ensembl ID', 'gene', 'HGNC ID']

    inhouse = inhouse[['gene', 'HGNC ID']].drop_duplicates().dropna()

    # edit NGNC field to have the same format as in GRCh37
    inhouse['HGNC ID'] = inhouse['HGNC ID'].apply(
        lambda x: x.replace('HGNC:', '') if x else x).astype(float) 

    return inhouse

def get_grch37_inhouse_map():
    """
        This function creates the mapping from GRCh37 gene names from ensembl into gencode v.34
        using HGNC-id as key
    """

    # load GRCh37
    dataset = Dataset(name='hsapiens_gene_ensembl',  host='http://grch37.ensembl.org')
    # attributes = dataset.list_attributes() -- look into attributes

    hg37_table = dataset.query(
        attributes=['hgnc_id','chromosome_name', 'hgnc_symbol'])

    # drop scaffolds not mapped to chromosomes
    hg37_table = hg37_table[~hg37_table['Chromosome/scaffold name'].apply(
        lambda x: x.startswith('CHR_') or 'H' in x or x.startswith('GL'))]

    # leave only mapping information
    hg37_table = hg37_table[['HGNC ID', 'HGNC symbol', 'Chromosome/scaffold name']].dropna().drop_duplicates()

    
    inhouse = get_gencode_v34_HGNC_genes()[['gene', 'HGNC ID']]
    
    mapping = hg37_table.rename(columns={'HGNC symbol': 'gene_grch37'}).merge(inhouse,  how='outer', on='HGNC ID')

    return mapping


def get_grch38_inhouse_map():
    """
        This function creates the mapping from GRCh37 gene names from ensembl into gencode v.34
        using HGNC-id as key
    """

    # load GRCh38
    dataset = Dataset(name='hsapiens_gene_ensembl',  host='http://ensembl.org')
    # attributes = dataset.list_attributes() -- look into attributes

    hg38_table = dataset.query(
        attributes=['hgnc_id','chromosome_name', 'hgnc_symbol'])

    # drop scaffolds not mapped to chromosomes
    hg38_table = hg38_table[~hg38_table['Chromosome/scaffold name'].apply(
        lambda x: x.startswith('CHR_') or 'H' in x or x.startswith('GL'))]

    # leave only mapping information
    hg38_table = hg38_table[['HGNC ID', 'HGNC symbol', 'Chromosome/scaffold name']].dropna().drop_duplicates()
    hg38_table['HGNC ID'] = hg38_table['HGNC ID'].apply(lambda x: x.replace('HGNC:', '') if x else x).astype(float)

    inhouse = get_gencode_v34_HGNC_genes()[['gene', 'HGNC ID']]
    
    mapping = hg38_table.rename(columns={'HGNC symbol': 'gene_grch38'}).merge(inhouse,  how='outer', on='HGNC ID')

    return mapping

All of the filters follows the same procedure:

1. Create mapping of `GRCh37` gene names from ensembl into `gencode v.34`

2. Read data of interest, that contains gene names in some format 

3. Identify genes from the previous step, that are not in `gencode v.34`

4. Create corretion map from genes in `step 3` to `gencode v.34` where possible by using mapping from `step 1`

5. Apply this correction map to our data of interest
    

# Gene panels

In [ ]:
# read original gene panel

gene_panels = pd.read_csv(".../450k/regions/gene-panel.txt", header=None)
gene_panels.columns = ['Gene name', 'panel']

print ("Number of genes:", gene_panels.shape[0])

In [ ]:
# GRCh37 to gencode v.34 mapping
grch37_gencode_v34 = get_grch37_inhouse_map()
gencode_v34_genes = set(grch37_gencode_v34['gene'].dropna().tolist())

grch37_gencode_v34 = grch37_gencode_v34.dropna()

# get genes not in gencode v.34 format
wrong_format_genes = gene_panels.loc[~gene_panels['Gene name'].isin(gencode_v34_genes), 'Gene name'].tolist()

# get a dict for correction
correcting_map = grch37_gencode_v34[grch37_gencode_v34['gene_grch37'].isin(wrong_format_genes)][['gene_grch37', 'gene']]
correcting_map = correcting_map.set_index('gene_grch37').to_dict()['gene']

# print information
print ("Number of genes not in gencode_34:", len(wrong_format_genes))
print ("Number of genes not in gencode_34, that found in GRCh37 :", len(correcting_map))

# manual add a missing case
correcting_map['KIF1BP'] = 'KIFBP'

print ("Full correction dictionary:", len(correcting_map))

# correct gene panels
gene_panels ['Gene name'] = gene_panels['Gene name'].apply(lambda x: correcting_map.get(x, x))

print ("Number of genes not in gencode_34:", sum(~gene_panels['Gene name'].isin(gencode_v34_genes)))

# # save dataset
gene_panels.to_csv(".../450k/regions/gene-panel-gencode-v34.txt", header=False, index=False, sep=',')


In [ ]:
! head -3 .../450k/regions/gene-panel-gencode-v34.txt

# S_hets

## Cassa

In [ ]:
cassa = pd.read_csv(".../450k/selection_cassa/cassa_supp_table_1.txt", sep='\t').drop_duplicates().dropna()
print ("Number of genes:", cassa.shape[0])

cassa.head(3)

In [ ]:
gene_panels = pd.read_csv(".../450k/regions/gene-panel.txt", header=None)
gene_panels = set(gene_panels[0].tolist())

print ("Genes in panel before correction:", len(gene_panels))
print ("Genes in cassa before correction", sum(cassa['gene_symbol'].isin(gene_panels)))

In [ ]:
# GRCh37 to gencode v.34 mapping
grch37_gencode_v34 = get_grch37_inhouse_map()
gencode_v34_genes = set(grch37_gencode_v34['gene'].dropna().tolist())
grch37_gencode_v34 = grch37_gencode_v34.dropna()

# get genes not in gencode v.34 format
wrong_format_genes = cassa.loc[~cassa['gene_symbol'].isin(gencode_v34_genes), 'gene_symbol'].tolist()

# get a dict for correction
correcting_map = grch37_gencode_v34[grch37_gencode_v34['gene_grch37'].isin(wrong_format_genes)][['gene_grch37', 'gene']]
correcting_map = correcting_map.set_index('gene_grch37').to_dict()['gene']

# print information
print ("Number of genes not in gencode_34:", len(wrong_format_genes))
print ("Number of genes not in gencode_34, that found in GRCh37 :", len(correcting_map))

# correct gene panels
cassa['gene_symbol'] = cassa['gene_symbol'].apply(lambda x: correcting_map.get(x, x))

print ("Number of genes not in gencode_34:", sum(~cassa['gene_symbol'].isin(gencode_v34_genes)))

# save dataset
cassa.to_csv(".../450k/selection_cassa/cassa_supp_table_1_gencode-v34.txt", index=False, sep='\t')

In [ ]:
gene_panels = pd.read_csv(".../450k/regions/gene-panel-gencode-v34.txt", header=None)
gene_panels = set(gene_panels[0].tolist())

print ("Genes in panel after correction:", len(gene_panels))
print ("Genes in cassa after correction", sum(cassa['gene_symbol'].isin(gene_panels)))

## Weghorn

In [ ]:
weghorn = pd.read_csv(".../450k/selection_weghorn/weghorn_supp_table1.txt", sep='\t')
print ("Number of genes:", weghorn.shape[0])

weghorn.head(3)

In [ ]:
gene_panels = pd.read_csv(".../450k/regions/gene-panel.txt", header=None)
gene_panels = set(gene_panels[0].tolist())

print ("Genes in panel before correction:", len(gene_panels))
print ("Genes in weghorn before correction", sum(weghorn['Gene'].isin(gene_panels)))

In [ ]:
# GRCh37 to gencode v.34 mapping
grch37_gencode_v34 = get_grch37_inhouse_map()
gencode_v34_genes = set(grch37_gencode_v34['gene'].dropna().tolist())
grch37_gencode_v34 = grch37_gencode_v34.dropna()


# get genes not in gencode v.34 format
wrong_format_genes = weghorn.loc[~weghorn['Gene'].isin(gencode_v34_genes), 'Gene'].tolist()

# get a dict for correction
correcting_map = grch37_gencode_v34[grch37_gencode_v34['gene_grch37'].isin(wrong_format_genes)][['gene_grch37', 'gene']]
correcting_map = correcting_map.set_index('gene_grch37').to_dict()['gene']

# print information
print ("Number of genes not in gencode_34:", len(wrong_format_genes))
print ("Number of genes not in gencode_34, that found in GRCh37 :", len(correcting_map))

# correct gene panels
weghorn['Gene'] = weghorn['Gene'].apply(lambda x: correcting_map.get(x, x))

print ("Number of genes not in gencode_34:", sum(~weghorn['Gene'].isin(gencode_v34_genes)))

# save dataset
weghorn.to_csv(".../450k/selection_weghorn/weghorn_supp_table1_gencode-v34.txt", index=False, sep='\t')


In [ ]:
gene_panels = pd.read_csv(".../450k/regions/gene-panel-gencode-v34.txt", header=None)
gene_panels = set(gene_panels[0].tolist())

print ("Genes in panel after correction:", len(gene_panels))
print ("Genes in weghorn after correction", sum(weghorn['Gene'].isin(gene_panels)))

In [ ]:
# save weghorn in a nice way
weghorn = pd.read_csv(".../450k/selection_weghorn/weghorn_supp_table1_gencode-v34.txt", sep='\t')

# mut-sel balance modenl and drift
weghorn_det = weghorn[['Gene', 's_het_det']].copy()
weghorn_drift = weghorn[['Gene', 's_het_drift']].copy()

# rename columns
weghorn_det.columns = ['gene_symbol', 's_het']
weghorn_drift.columns = ['gene_symbol', 's_het']

# save
weghorn_det.to_csv(".../450k/selection_weghorn/weghorn_det_gencode-v34.txt", sep='\t', index=False)
weghorn_drift.to_csv(".../450k/selection_weghorn/weghorn_drift_gencode-v34.txt", sep='\t', index=False)

## PLI

In [ ]:
! wget https://storage.googleapis.com/gcp-public-data--gnomad/release/2.1.1/constraint/gnomad.v2.1.1.lof_metrics.by_gene.txt.bgz
! mv ./gnomad.v2.1.1.lof_metrics.by_gene.txt.bgz .../450k/selection_pli/gnomad.v2.1.1.lof_metrics.by_gene.txt.bgz 

In [ ]:
gnomad_filepath = '.../450k/selection_pli/gnomad.v2.1.1.lof_metrics.by_gene.txt.bgz'

gnomad = pd.read_csv(gnomad_filepath, compression='gzip', sep='\t')

gnomad = gnomad[['gene', 'pLI']]
gnomad = gnomad[~gnomad['pLI'].isnull()]
gnomad.columns = ['gene_symbol', 's_het']

gnomad.head(3)

In [ ]:
gene_panels = pd.read_csv(".../450k/regions/gene-panel.txt", header=None)
gene_panels = set(gene_panels[0].tolist())

print ("Genes in panel before correction:", len(gene_panels))
print ("Genes in gnomAD before correction", sum(gnomad['gene_symbol'].isin(gene_panels)))

In [ ]:
# GRCh37 to gencode v.34 mapping
grch37_gencode_v34 = get_grch37_inhouse_map()
gencode_v34_genes = set(grch37_gencode_v34['gene'].dropna().tolist())
grch37_gencode_v34 = grch37_gencode_v34.dropna()


# get genes not in gencode v.34 format
wrong_format_genes = gnomad.loc[~gnomad['gene_symbol'].isin(gencode_v34_genes), 'gene_symbol'].tolist()

# get a dict for correction
correcting_map = grch37_gencode_v34[grch37_gencode_v34['gene_grch37'].isin(wrong_format_genes)][['gene_grch37', 'gene']]
correcting_map = correcting_map.set_index('gene_grch37').to_dict()['gene']

# print information
print ("Number of genes not in gencode_34:", len(wrong_format_genes))
print ("Number of genes not in gencode_34, that found in GRCh37 :", len(correcting_map))

# correct gene panels
gnomad['gene_symbol'] = gnomad['gene_symbol'].apply(lambda x: correcting_map.get(x, x))

print ("Number of genes not in gencode_34:", sum(~gnomad['gene_symbol'].isin(gencode_v34_genes)))


In [ ]:
print ("Gnomad shape:", gnomad.shape)
print ("Gnomad unique genes:", gnomad['gene_symbol'].drop_duplicates().shape)

gnomad = gnomad.groupby('gene_symbol').max().reset_index()

print ("Gnomad shape after correction:", gnomad.shape)

In [ ]:
# save dataset
gnomad.to_csv(".../450k/selection_pli/gnomad.v2.1.1.PLI_gencode-v34.txt", index=False, sep='\t')

In [ ]:
gene_panels = pd.read_csv(".../450k/regions/gene-panel-gencode-v34.txt", header=None)
gene_panels = set(gene_panels[0].tolist())

print ("Genes in panel after correction:", len(gene_panels))
print ("Genes in gnomAD after correction", sum(gnomad['gene_symbol'].isin(gene_panels)))

## Roulette

In [ ]:
roulette = pd.read_csv(".../450k/selection_roulette/s_het_roulette.csv", sep=',').rename(
    columns={'mean_s_het': 's_het'})
roulette = roulette[['Gene_symbol', 's_het']].drop_duplicates().dropna()
roulette = roulette.groupby('Gene_symbol').mean().reset_index()

print ("Number of genes:", roulette.shape[0])

roulette.head(3)

In [ ]:
gene_panels = pd.read_csv(".../450k/regions/gene-panel.txt", header=None)
gene_panels = set(gene_panels[0].tolist())

print ("Genes in panel before correction:", len(gene_panels))
print ("Genes in roulette before correction", sum(roulette['Gene_symbol'].isin(gene_panels)))

In [ ]:

# GRCh38 to gencode v.34 mapping
grch38_gencode_v34 = get_grch38_inhouse_map()
gencode_v34_genes = set(grch38_gencode_v34['gene'].dropna().tolist())
grch38_gencode_v34 = grch38_gencode_v34.dropna()

# get genes not in gencode v.34 format
wrong_format_genes = roulette.loc[~roulette['Gene_symbol'].isin(gencode_v34_genes), 'Gene_symbol'].tolist()

# get a dict for correction
correcting_map = grch38_gencode_v34[grch38_gencode_v34['gene_grch38'].isin(wrong_format_genes)][['gene_grch38', 'gene']]
correcting_map = correcting_map.set_index('gene_grch38').to_dict()['gene']

# print information
print ("Number of genes not in gencode_34:", len(wrong_format_genes))
print ("Number of genes not in gencode_34, that found in GRCh38 :", len(correcting_map))

# correct gene panels
roulette['Gene_symbol'] = roulette['Gene_symbol'].apply(lambda x: correcting_map.get(x, x))

print ("Number of genes not in gencode_34:", sum(~roulette['Gene_symbol'].isin(gencode_v34_genes)))

# save dataset
roulette = roulette.rename(columns={'Gene_symbol': 'gene_symbol'})
roulette.to_csv(".../450k/selection_roulette/s_het_roulette_gencode-v34.csv", index=False, sep='\t')


In [ ]:
gene_panels = pd.read_csv(".../450k/regions/gene-panel-gencode-v34.txt", header=None)
gene_panels = set(gene_panels[0].tolist())

print ("Genes in panel after correction:", len(gene_panels))
print ("Genes in roulette after correction", sum(roulette['gene_symbol'].isin(gene_panels)))

### High s-het genes stat for paper

In [ ]:
import numpy as np

gene_panels = pd.read_csv(".../450k/regions/gene-panel-gencode-v34.txt", header=None)
gene_panels.columns = ['gene_symbol', 'panel']
gene_panels['n_genes'] = gene_panels.groupby('panel')['gene_symbol'].transform('count')

roulette_recessive = roulette.merge(gene_panels, how='inner', on='gene_symbol')
roulette_recessive ['high_s_het'] = roulette_recessive['s_het'] >= 0.1

print ("Number of recessive genes in panel:", roulette_recessive.shape[0])
print (f"Number of high s-het genes: {roulette_recessive ['high_s_het'].sum()}")

In [ ]:
roulette_recessive = roulette_recessive.groupby('panel').agg({'n_genes': 'first', 'gene_symbol': 'count', 'high_s_het': 'sum'}).reset_index()

roulette_recessive = roulette_recessive.sort_values(by='n_genes', ascending=False).rename(
    columns={'gene_symbol': 'n_genes_with_s_het', 'high_s_het': 'n_high_s_het'})

roulette_recessive['n_genes_with_s_het_percentage'] = np.round(roulette_recessive['n_genes_with_s_het'] / roulette_recessive['n_genes'] * 100, 1)
roulette_recessive['high_s_het_prcentage'] = np.round(roulette_recessive['n_high_s_het'] / roulette_recessive['n_genes_with_s_het'] * 100, 1)

roulette_recessive['number_of_genes_with_s_het'] = roulette_recessive['n_genes_with_s_het'].astype(str) + ' (' + roulette_recessive['n_genes_with_s_het_percentage'].astype(str) + '%)'
roulette_recessive['number_of_high_s_het_genes'] = roulette_recessive['n_high_s_het'].astype(str) + ' (' + roulette_recessive['high_s_het_prcentage'].astype(str) + '%)'

roulette_recessive.to_excel('../../../data/tables/high_s_het_genes_roulette.xlsx', index=False)